In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [5]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [7]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [9]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [11]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [13]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [15]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

In [17]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [19]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, you’re trying to stump me with basic math! Well, the answer is 4. Shocking, I know.


In [21]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [23]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [25]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [27]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nThe website serves as a personal platform for Ed, who is a developer specializing in Large Language Models (LLMs) and artificial intelligence (AI). He is the co-founder and CTO of Nebula.io, a company focused on AI solutions for talent acquisition. Ed also has experience as the founder of another AI startup, untapt. His interests include coding, DJing, electronic music, and engaging with online tech communities.\n\n## Recent Announcements\n- **May 28, 2025:** Course announcement aimed at transforming individuals into LLM experts and leaders.\n- **May 18, 2025:** Announcement of a 2025 AI Executive Briefing.\n- **April 21, 2025:** Launch of "The Complete Agentic AI Engineering Course."\n- **January 23, 2025:** Resources shared for an "LLM Workshop – Hands-on with Agents." \n\nFor updates and to engage further, visitors are encouraged to connect with Ed through various social media platforms and his newsletter.'

In [29]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [31]:
display_summary("https://edwarddonner.com")

# Summary of the Edward Donner Website

Edward Donner is an enthusiast and expert in coding, LLMs (Large Language Models), DJing, and electronic music production. He is the co-founder and CTO of [Nebula.io](http://Nebula.io), a company leveraging AI to enhance talent discovery and management in organizations. He previously founded the AI startup untapt, which was acquired in 2021. The website features various posts about his activities, interests, and announcements related to LLM education and workshops.

## Announcements
- **May 28, 2025**: Course on becoming an LLM expert and leader.
- **May 18, 2025**: Announcement of the 2025 AI Executive Briefing.
- **April 21, 2025**: Introduction of the Complete Agentic AI Engineering Course.
- **January 23, 2025**: Announcement of a hands-on LLM Workshop focused on agents.

The website encourages connections and offers insights into Ed's work and interests in AI and technology.

In [33]:
display_summary("https://cnn.com")

# Summary of CNN Website

CNN is a leading news outlet that provides the latest updates on global events, politics, business, health, entertainment, sports, science, and climate. The website features a variety of sections to cater to diverse interests, including:

- **Breaking News**: Current global developments, including ongoing conflicts such as the Ukraine-Russia War and the Israel-Hamas conflict.
- **Politics**: In-depth analyses of Trump administration policies, including recent tariff threats against countries like India for importing Russian oil. 
- **Health & Wellness**: Discussions on public health topics, including childhood abuse impacts and updates on global vaccination efforts against polio.
- **Entertainment**: Coverage of celebrity news and cultural events, including recent deaths in the entertainment industry and major award nominations.
- **Business & Tech**: Updates on significant financial investments and technological advancements, such as Apple's investments in the U.S.
- **Science & Discovery**: Reporting on NASA missions and advancements in robotics and environmental conservation efforts.

## Recent Highlights
- **Trump's Tariff Threat**: President Trump announced new tariffs on India in response to Russian oil imports, which may impact both economies.
- **Israel-Hamas War Updates**: Israel’s military chief advises against a full takeover of Gaza amidst ongoing discussions on military strategy.
- **Ukraine-Russia Conflict**: Continuous developments in the geopolitical climate regarding Ukraine and Russia.
- **Severe Weather Impacts**: Reports of devastating flooding in Northern India that has resulted in multiple casualties and evacuations.

CNN aims to provide an extensive range of information and insights to keep its audience informed on major global issues.